## Importing Required libraries

In [152]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys
import time

In [153]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [184]:
browser = webdriver.Chrome(r'C:\Users\ADMIN\Downloads\chromedriver.exe')
browser.get('https://agmarknet.gov.in/default.aspx') # Webdriver accessing given website

In [185]:
# Putting relevant filters to fetch the Potato prices for Agra in 2020

browser.find_element_by_xpath('//*[@id="ddlArrivalPrice"]').send_keys('Price')
time.sleep(1)

browser.find_element_by_xpath('//*[@id="ddlCommodity"]').send_keys('Potato')
time.sleep(1)

browser.find_element_by_xpath('//*[@id="ddlState"]').send_keys('Uttar Pradesh')
time.sleep(1)

browser.find_element_by_xpath('//*[@id="ddlDistrict"]').send_keys('Agra')
time.sleep(1)

browser.find_element_by_xpath('//*[@id="txtDate"]').clear()
time.sleep(1)

browser.find_element_by_xpath('//*[@id="txtDate"]').send_keys('01-Jan-2020')
time.sleep(1)

browser.find_element_by_xpath('//*[@id="txtDateTo"]').clear()
time.sleep(1)

browser.find_element_by_xpath('//*[@id="txtDateTo"]').send_keys('31-Dec-2020')

browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btnGo"]'))))

In [186]:
result_dic ={}
column_names = ['S_Number', 'District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'Min Price (Rs./Quintal)', 
                'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)', 'Price Date']
for column in column_names:
    result_dic[column] = result_dic.get(column,[])       

{'S_Number': [],
 'District Name': [],
 'Market Name': [],
 'Commodity': [],
 'Variety': [],
 'Grade': [],
 'Min Price (Rs./Quintal)': [],
 'Max Price (Rs./Quintal)': [],
 'Modal Price (Rs./Quintal)': [],
 'Price Date': []}

In [187]:
def tabular_data():
    table = browser.find_element_by_xpath('//*[@id="cphBody_GridPriceData"]') # Accessing table element

    #Iterating through each row in table 

    for row in table.find_elements_by_xpath('//*[@id="cphBody_GridPriceData"]/tbody/tr'):
        if row.text != '':
            row_values = row.text
            if row_values[0] != 'S':
                row_values = row_values.split(' ')
                result_dic['S_Number'] += [row_values[0]]
                result_dic['District Name'] += [row_values[1]]
                result_dic['Market Name'] += [row_values[2]]
                result_dic['Commodity'] += [row_values[3]]
                result_dic['Variety'] += [row_values[4]]
                result_dic['Grade'] += [row_values[5]]
                result_dic['Min Price (Rs./Quintal)'] += [row_values[6]]
                result_dic['Max Price (Rs./Quintal)'] += [row_values[7]]
                result_dic['Modal Price (Rs./Quintal)'] += [row_values[8]]
                result_dic['Price Date'] += ['-'.join(row_values[9:])]

    

In [188]:
tabular_data()

In [189]:
browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cphBody_GridPriceData"]/tbody/tr[52]/td/table/tbody/tr/td[1]/input'))))

In [190]:
tabular_data() 

In [191]:
while True:
    try:
        browser.execute_script("arguments[0].click();", WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cphBody_GridPriceData"]/tbody/tr[52]/td/table/tbody/tr/td[3]/input'))))
        time.sleep(20)
        tabular_data()
    except TimeoutException:
        break
    

In [196]:
result_df = pd.DataFrame(result_dic)
result_df.set_index('S_Number', inplace=True)